In [ ]:
# define logging and working directory
import os
# define GPU id to use
# 0 = 1080 Bus ID 2
# 1 = Titan Bus ID 131
# 2 = Titan Bus ID 132
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
current_gpu = '/device:GPU:0'


import logging
import json
import platform
import SimpleITK as sitk
import glob
import datetime
import random
from collections import Counter
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
from medpy.metric.binary import hd, dc,jc,precision,recall
import tensorflow as tf
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from ipywidgets import interact
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# setup working directory
print(os.getcwd())
# change working directory to project root - neccessary for jupyter notebooks
if platform.system() is "Windows":
    os.chdir('E:\\sven\\git\\cardio')
else:
    os.chdir('/Users/minority/Code/Git/cardio')
print(os.getcwd())


from src.utils.utils_io import Console_and_file_logger, ensure_dir, save_config
from src.utils.myshow import myshow, myshow3d
from src.visualization.visualize import plot_3d_vol, plot_4d_vol
from src.data.dataset import get_metadata_maybe, filter_4d_vol, describe_sitk
from src.data.generators import DataGenerator3D
from src.data.dataset import get_3d_img_msk_files
from src.utils.my_metrics import jaccard_coef, jaccard_coef_background, jaccard_coef_rv, jaccard_coef_lv, jaccard_coef_myo, bce_dice_iou_loss, weighted_categorical_crossentropy, cce_dice_loss, weighted_cce_dice_coef
from src.utils.unet_3d_metrics import weighted_dice_coefficient_loss
from src.models.ModelManager import create_3D_unet
from src.utils.KerasCallbacks import TrainValTensorBoard, CustomImageWriter3D



# define experiment name for report, model and log paths + filenames
EXPERIMENT = '3D_unet_cce_dice_loss'
now = datetime.datetime.now()



In [13]:
# define logging and working directory
# define logging and working directory
import os

import logging
import platform
print(os.getcwd())
# change working directory to project root - neccessary for jupyter notebooks
if platform.system() is "Windows":
    os.chdir('E:\\sven\\git\\cardio')
else:
    os.chdir('/Users/minority/Code/Git/cardio')
print(os.getcwd())

# define GPU id to use
# 0 = 1080 Bus ID 2
# 1 = Titan Bus ID 131
# 2 = Titan Bus ID 132
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
current_gpu = '/device:GPU:0'

from src.utils.utils_io import Console_and_file_logger, ensure_dir, save_config
from src.utils.myshow import myshow, myshow3d
import src.utils.my_metrics as metr
import src.utils.medley_metrics as m_metrics
from src.data.dataset import load_data, load_acdc_data
from src.utils.Evaluation import sanity_check
from src.visualization.visualize import plot_confusion_matrix, show_slice

from src.visualization.visualize import plot_3d_vol, plot_4d_vol
from src.data.dataset import get_metadata_maybe, filter_4d_vol, describe_sitk
from src.data.generators import DataGenerator3D
from src.data.dataset import get_3d_img_msk_files

from medpy.metric.binary import hd, dc,jc,precision,recall

Console_and_file_logger('Evaluation', logging.INFO)

import glob
import random
from collections import Counter

import keras
import tensorflow as tf
from keras.models import model_from_json

from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from numpy import newaxis
import json

from ipywidgets import interact
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

metrics = [
    metr.jaccard_coef,
    metr.jaccard_coef_background,
    metr.jaccard_coef_myo,
    metr.jaccard_coef_lv,
    metr.jaccard_coef_rv,
]



E:\sven\git\cardio
E:\sven\git\cardio


In [8]:
@interact
def load_config(config_file=glob.glob('reports/configs/**/**/*.json', recursive=False), load=False):
    """
    load config, tranings history and model
    """
    # load config with all params into global namespace

    with open(config_file, encoding='utf-8') as data_file:
        config = json.loads(data_file.read())
    #globals().update(config)
    globals()['config'] = config
    logging.info('Experiment: {}'.format(config['EXPERIMENT']))
    logging.info('config:\n {}'.format(json.dumps(config, indent=4, sort_keys=True)))
    Console_and_file_logger(EXPERIMENT, logging.INFO)


    # load all experiment files if user asked for it
    if load:

        glob_ = {}
        try:
            # load trainings history
            logging.info('loading trainings history...')
            glob_['df_history'] = pd.read_csv(os.path.join(HISTORY_PATH, EXPERIMENT + '.csv'),index_col=0)
            logging.info('history {} loaded'.format(os.path.join(HISTORY_PATH, EXPERIMENT + '.csv')))
        except Exception as e:
            logging.info(str(e))

        try:
            # load model
            logging.info('loading model...')
            model = model_from_json(open(os.path.join(MODEL_PATH, 'model.json')).read())
            model.load_weights(os.path.join(MODEL_PATH, 'checkpoint.h5'))
            model.compile(optimizer=OPTIMIZER, loss=keras.losses.categorical_crossentropy, metrics=metrics)
            glob_['model'] = model
            logging.info('model {} loaded'.format(os.path.join(MODEL_PATH, 'model.json')))
        except Exception as e:
            logging.info(str(e))
        

        try:
            # load past evaluations done with that model
            logging.info('loading past evaluation scores...')
            glob_['evaluation_score'] = pd.read_csv(os.path.join('reports/evaluation/', EXPERIMENT + '.csv')).set_index('Evaluation')
            logging.info('past evaluation scores {} loaded'.format(os.path.join('reports/evaluation/', EXPERIMENT + '.csv')))
        except Exception as e:
            # delete the evaluation score object from current namespace if past models & evaluations have beend done
            globals().pop('evaluation_score', None)
            logging.info(str(e))
        # update global namesspace for further tests
        globals().update(glob_)

interactive(children=(Dropdown(description='config_file', options=('reports/configs\\2d_unet_acdc\\2019-03-09_…

In [14]:
x_train_f, y_train_f = get_3d_img_msk_files('data/raw/tetra/3D/train/')
x_val_f, y_val_f = get_3d_img_msk_files('data/raw/tetra/3D/val/')
x_test_f, y_test_f = get_3d_img_msk_files('data/raw/tetra/3D/test/')

logging.info('x_train files: {}, y_train files: {}'.format(len(x_train_f), len(y_train_f)))
logging.info('x_val files: {}, y_val files: {}'.format(len(x_val_f), len(y_val_f)))
logging.info('x_test files: {}, y_test files: {}'.format(len(x_test_f), len(y_test_f)))

# create a batch generator, filter size, for faster testing
batch_generator = DataGenerator3D(x_train_f, y_train_f, batch_size=config['BATCHSIZE'], dim=(config['IMG_Z'], config['IMG_WIDTH'], config['IMG_HEIGHT']), spacing=(1.25,1.25,8), mask_values=MASK_VALUES)
validation_generator = DataGenerator3D(x_val_f, y_val_f, batch_size=config['BATCHSIZE'], dim=(config['IMG_Z'], config['IMG_WIDTH'], config['IMG_HEIGHT']), spacing=(1.25,1.25,8), mask_values=MASK_VALUES)
test_generator = DataGenerator3D(x_test_f, y_test_f, dim=(config['IMG_Z'], config['IMG_WIDTH'], config['IMG_HEIGHT']), spacing=(1.25,1.25,8), mask_values=MASK_VALUES)



2019-04-26 18:49:41,827 INFO x_train files: 757, y_train files: 757
2019-04-26 18:49:41,827 INFO x_val files: 126, y_val files: 126
2019-04-26 18:49:41,827 INFO x_test files: 127, y_test files: 127
2019-04-26 18:49:41,827 INFO Create DataGenerator
2019-04-26 18:49:41,827 INFO No augmentation
2019-04-26 18:49:41,827 INFO Create DataGenerator
2019-04-26 18:49:41,827 INFO No augmentation
2019-04-26 18:49:41,827 INFO Create DataGenerator
2019-04-26 18:49:41,827 INFO No augmentation


In [15]:
# predict for all batches
from medpy.metric.binary import hd, dc,jc,precision,recall
import numpy as np
gt_ = []
pred_ = []
img_ = []
n_batches = len(test_generator)
#n_batches = 2
logging.info('load and predict {} batches'.format(n_batches))
#with tf.device(current_gpu):
#    pred = model.predict_generator(batch_generator, steps = len(batch_generator))
#
#gt_ = [gt_.extend(batch[1]) for batch in batch_generator]
    


for idx,batch in enumerate(test_generator):
    if idx <= n_batches:
        with tf.device(current_gpu):
            img_.extend(batch[0])
            pred_.extend(((model.predict_on_batch(batch[0]))>=0.5).astype(np.bool))
        gt_.extend(batch[1])
    else:
        break

# reshape
gt = np.array(gt_)
pred = np.array(pred_)

logging.info('gt shape: {}'.format(gt.shape))
logging.info('pred shape: {}'.format(pred.shape))
del gt_
del pred_

2019-04-26 18:49:47,225 INFO load and predict 127 batches
2019-04-26 18:52:08,397 INFO gt shape: (128, 12, 256, 256, 4)
2019-04-26 18:52:08,412 INFO pred shape: (128, 12, 256, 256, 4)


In [16]:
# calc IOU per channel
for c in range(pred.shape[-1]):
    pred_ = pred[...,c]
    gt_ = gt[...,c]
    # calc medpy scores
    jaccard_coef = jc(pred_, gt_)
    dice_score = dc(pred_, gt_)
    precision_score = precision(pred_, gt_)
    recall_score = recall(pred_, gt_)

    logging.info('jac: {}'.format(jaccard_coef))
    logging.info('dice: {}'.format(dice_score))
    logging.info('prec: {}'.format(precision_score))
    logging.info('recall: {}'.format(recall_score))




2019-04-26 18:52:16,120 INFO jac: 0.9945014631047002
2019-04-26 18:52:16,120 INFO dice: 0.9972431522377825
2019-04-26 18:52:16,120 INFO prec: 0.9974677345929726
2019-04-26 18:52:16,120 INFO recall: 0.9970186709903857
2019-04-26 18:52:19,150 INFO jac: 0.8124138991646308
2019-04-26 18:52:19,150 INFO dice: 0.8964993035410783
2019-04-26 18:52:19,150 INFO prec: 0.8901523341423578
2019-04-26 18:52:19,150 INFO recall: 0.9029374333290774
2019-04-26 18:52:22,114 INFO jac: 0.6954737983400961
2019-04-26 18:52:22,114 INFO dice: 0.8203887303017946
2019-04-26 18:52:22,114 INFO prec: 0.7650892389977327
2019-04-26 18:52:22,114 INFO recall: 0.8843049528235855
2019-04-26 18:52:25,268 INFO jac: 0.8747032837333085
2019-04-26 18:52:25,268 INFO dice: 0.9331645080296792
2019-04-26 18:52:25,268 INFO prec: 0.9483716122535772
2019-04-26 18:52:25,268 INFO recall: 0.9184373978424322


In [17]:
@interact
def evaluate_with_path(path='data/raw/tetra/3D/test/',start=False,):
    """
    Run model evaluation with all compiled metrics for a given directory
    """
    if start:
        path_as_string = path.replace('/','_')
        
        # load data sliced
        x_train_f, y_train_f = get_3d_img_msk_files(path)

        # predict testdata
        test_generator = DataGenerator3D(x_train_f, y_train_f, batch_size=config['BATCHSIZE'], dim=(config['IMG_Z'], config['IMG_WIDTH'], config['IMG_HEIGHT']), spacing=(1.25,1.25,8), mask_values=MASK_VALUES)

        with tf.device(current_gpu):
            scores = model.evaluate_generator(test_generator, max_queue_size=10, verbose=1)

        
        # get, or create a dataframe from results, and store results in global namespace
        col = ['Evaluation'] +  model.metrics_names
        row = scores
        # get evaluation score from global namespace, or create a new one
        scores_df = globals().get('evaluation_score', pd.DataFrame(columns=col).set_index('Evaluation'))
        scores_df.loc[path_as_string] = row
        
                                  
        globals()['evaluation_score'] = scores_df
        export_path = os.path.join('reports/evaluation', EXPERIMENT)
        ensure_dir(export_path)
        scores_df.to_csv(os.path.join(export_path, 'metrics_' + path_as_string + '.csv'))
        return scores_df

interactive(children=(Text(value='data/raw/tetra/3D/test/', description='path'), Checkbox(value=False, descrip…

In [19]:
del evaluation_score